In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.model_selection import train_test_split
eps=np.finfo(float).eps
from binarytree import tree,Node
from sklearn.metrics import classification_report, confusion_matrix ,accuracy_score
from sklearn.tree import DecisionTreeClassifier
from operator import itemgetter
import copy
import math

In [2]:
df=pd.read_csv("train.csv")

In [3]:
X =df.drop(['left'],axis=1)
# X = pd.concat([X,pd.get_dummies(df['sales'], prefix='sales')],axis=1)
# X = pd.concat([X,pd.get_dummies(df['salary'], prefix='salary')],axis=1)
y=df['left']
numeric=['last_evaluation','satisfaction_level','average_montly_hours','time_spend_company','number_project']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
orignalX=copy.deepcopy(X_train)
orignalY=copy.deepcopy(y_train)
testX=copy.deepcopy(X_test)
testY=copy.deepcopy(y_test)

orignalXEntropy=copy.deepcopy(X_train)
orignalYEntropy=copy.deepcopy(y_train)
testXEntropy=copy.deepcopy(X_test)
testYEntropy=copy.deepcopy(y_test)

orignalXGini=copy.deepcopy(X_train)
orignalYGini=copy.deepcopy(y_train)
testXGini=copy.deepcopy(X_test)
testYGini=copy.deepcopy(y_test)

orignalXCR=copy.deepcopy(X_train)
orignalYCR=copy.deepcopy(y_train)
testXCR=copy.deepcopy(X_test)
testYCR=copy.deepcopy(y_test)


In [5]:
def Gini(OrignalXGini,OrignalYGini,testXGini,testYGini):
    def findParentGini(df):
        if df.empty==True:
            return
        classCol=df.keys()[-1]
        #yeh sirf header return karega
        resultValues=df[classCol].unique()
        #all ouput values in output column
        gini=2
        for value in resultValues:
            fraction=df[classCol].value_counts()[value]/(len(df[classCol])+eps)
            gini*=fraction
        return abs(gini)
    def findRelativeGini(df,attribute):
        if df.empty==True:
            return
        classCol= df.keys()[-1] 
        #last col assumed as result 
        #getting target class 
        resultValues=df[classCol].unique() 
        attributeNames=df[attribute].unique()
        gini2 = 0
        for attr in attributeNames:
            gini = 2
            for value in resultValues:
                #temp&hot with yes
                num = len(df[attribute][df[attribute]==attr][df[classCol]==value])
                #total hot
                den = len(df[attribute][df[attribute]==attr])
                fraction = num/(den+eps)
                gini *= fraction
            #entropy attribute/total*(entropy attribute_values)
            fraction2 = den/len(df)  
            gini2 += fraction2*gini
        return abs(gini2)  
    def get_subtable(df, node,value):
        return df[df[node] == value].reset_index(drop=True)
    def get_subtableSmaller(df, node,value):
        if df.empty==True:
            return
        return df[df[node] < value].reset_index(drop=True)

    def get_subtableGreater(df, node,value):
        if df.empty==True:
            return
        return df[df[node] >= value].reset_index(drop=True)

    def bestAttributeGini(df):
        if df.empty==True:
            return
        infoGain={}
        max2=0
        attr=""
        entropy_1=findParentGini(df)
        for key in df.keys()[:-1]:

            if key not in numeric:
    #             print("XXX-------",key)
                entropy_2=findRelativeGini(df,key)
                if ((entropy_1-entropy_2)) > max2:
                    max2=entropy_1-entropy_2
                    attr=key
        t=(max2,0)
        infoGain[attr]=t
        return infoGain
    


    df=pd.concat([OrignalXGini,OrignalYGini],axis=1)
    def split(df):

        splitpoint={}
        for col in numeric:
#             df.sort_values(col,inplace=True)
            prev=None;
            value=0;
            max1=0;
#             print(col)
            entropy1=findParentGini(df)
#         print(col)
            l=df[col].unique()
            for mid in l:
                subtableLeft=get_subtableSmaller(df,col,mid)
                subtableRight=get_subtableGreater(df,col,mid)
                fraction=len(subtableLeft)/len(df)
                fraction1=len(subtableRight)/len(df)
                firstSubtable=0
                secondSubtable=0
                if subtableLeft.empty:
                    firstSubtable=0
                else:
                    firstSubtable=fraction*findParentGini(subtableLeft)
    #                 print(subtableRight)
                if subtableRight.empty:
                    secondSubtable=0
                else:
                    secondSubtable=fraction1*findParentGini(subtableRight)

                entropy2=firstSubtable+secondSubtable
                ig1=abs(entropy1)-abs(entropy2)

                if ig1>max1:
                    max1=ig1
                    value=mid
            t=(max1,value)
            splitpoint[col]=t

        return splitpoint
      


    class node:
        def __init__(self,value):
            self.value=value
            self.child=[]
            self.edges=[]
            self.positive=0
            self.negative=0


    def buildTree(df):
        if len(df.columns)==1:
            return
        resultLabel= df.keys()[-1] 
        clValue,counts = np.unique(df[resultLabel],return_counts=True)

    #     print(clValue,counts)
        if(len(counts)==0):
            return 
        
        if len(counts)<=1:
            root=node(clValue[0])
            if clValue[0]==0:
                root.negative=counts[0]
            else:
                root.positive=counts[0]
    #         print("leaf")

        else:

            newsplit=split(df)
            calsplit=bestAttributeGini(df)
    #         finalSplit=newsplit.append(calsplit)
    #         for 
#             print(calsplit)
            newsplit.update(calsplit)
#             print(newsplit)
            maxig=0
            spoint=0
            attr=""
            for k,t1 in newsplit.items():
    #             t1=i.value()
                if maxig<t1[0]:
                    maxig=t1[0]
                    spoint=t1[1]
                    attr=k
    #         print(attr)
    #         print(type(spoint),spoint)
    #         attribute_name=bestAttribute(df)
            if attr=="":
                return None
        
            root=node(attr)
            root.split=spoint
            root.positive=counts[1]
            root.negative=counts[0]


    #         for value in attValue:
    #             print(type(value))

            if attr in numeric:
                subtable0 = get_subtableSmaller(df,attr,spoint)
        #         subtable0 =subtable0.drop([attribute_name],axis=1)
                subtable1 = get_subtableGreater(df,attr,spoint)
                root.child.append(buildTree(subtable0))
                root.child.append(buildTree(subtable1))

            else:
    #             k=0
    #             print(attr)
    #             print(df[attr].unique())
    #             return
#                 print("attri.......",attr)
                for j in df[attr].unique():
#                     print(j)
                    subtable0 = get_subtable(df,attr,j)
                    subtable0 =subtable0.drop([attr],axis=1)
                    root.edges.append(j)
                    root.child.append(buildTree(subtable0))


        return root


    # In[191]:


    # print(df)
    root=buildTree(df)
#     print(root.value)


    def testing(row1,root):
            if root==None:
                y1.append(0)
                return
            
            if len(root.child)==0:
                y1.append(root.value)
                return

            i=root.value
    #         if i in row1:
            num=0;
            if i in numeric:
                if row1[i]<root.split and root.child[0]!=None:
                    num=1;
                    testing(row1,root.child[0])
                elif row1[i]>=root.split and root.child[1]!=None:
                    num=1;
                    testing(row1,root.child[1])
                if num==0:
                    if root.positive>root.negative:
                        y1.append(1)
                    else:
                        y1.append(0)
                    return
                    
            else:
                k=0
                flag=0
                for j in root.edges:
                    if row1[i]==j and root.child[k]!=None:
#                         print("inside")
                        flag=1
                        testing(row1,root.child[k])
                        break
                    elif row1[i]==j:
                        if root.positive>root.negative:
                            y1.append(1)
                        else:
                            y1.append(0)
                        return
                    k=k+1
                if flag==0:
                    if root.positive>root.negative:
                        y1.append(1)
                    else:
                        y1.append(0)
                    return


    def testingData(df):
        for index,row in df.iterrows():
            testing(row,root)

    y1=[] 
    testingData(testXGini)  
#     print(len(y1))

    print(confusion_matrix(y_test, y1))  
    print(accuracy_score(y_test,y1))
    print(classification_report(y_test, y1)) 

In [6]:
def Entropy(orignalXEntropy,orignalYEntropy,testXEntropy,testYEntropy):
 




    numeric=['last_evaluation','satisfaction_level','average_montly_hours','time_spend_company','number_project']


    # In[181]:


    def findParentEntropy(df):
        if df.empty==True:
            return
        classCol=df.keys()[-1]
        #yeh sirf header return karega
        resultValues=df[classCol].unique()
        #all ouput values in output column
        entropy=0
        for value in resultValues:
            fraction=df[classCol].value_counts()[value]/(len(df[classCol])+eps)
            entropy+=-fraction*np.log2(fraction+eps)
        return abs(entropy)




    # In[182]:


    def findRelativeEntropy(df,attribute):
        if df.empty==True:
            return
        classCol= df.keys()[-1] 
        #last col assumed as result 
        #getting target class 
        resultValues=df[classCol].unique() 
        attributeNames=df[attribute].unique()
        entropy2 = 0
        for attr in attributeNames:
            entropy = 0
            for value in resultValues:
                #temp&hot with yes
                num = len(df[attribute][df[attribute]==attr][df[classCol]==value])
                #total hot
                den = len(df[attribute][df[attribute]==attr])
                fraction = num/(den+eps)
                entropy += -fraction*np.log2(fraction+eps)
            #entropy attribute/total*(entropy attribute_values)
            fraction2 = den/len(df)  
            entropy2 += -fraction2*entropy 
        return abs(entropy2)


    # In[183]:


    def get_subtable(df, node,value):
        return df[df[node] == value].reset_index(drop=True)


    # In[184]:


    def get_subtableSmaller(df, node,value):
        if df.empty==True:
            return
        return df[df[node] < value].reset_index(drop=True)


    # In[185]:


    def get_subtableGreater(df, node,value):
        if df.empty==True:
            return
        return df[df[node] >= value].reset_index(drop=True)


    # In[186]:


    def bestAttribute(df):
        if df.empty==True:
            return
        infoGain={}
        max2=0
        attr=""
        entropy_1=findParentEntropy(df)
        for key in df.keys()[:-1]:

            if key not in numeric:
    #             print("XXX-------",key)
                entropy_2=findRelativeEntropy(df,key)
                if ((entropy_1-entropy_2)) > max2:
                    max2=entropy_1-entropy_2
                    attr=key
        t=(max2,0)
        infoGain[attr]=t
        return infoGain


    # In[187]:



    df=pd.concat([orignalXEntropy,orignalYEntropy],axis=1)
    def split(df):

        splitpoint={}
        entropy1=findParentEntropy(df)
        for col in numeric:
            # df.sort_values(col,inplace=True)
            prev=None;
            value=0;
            max1=0;
#             print(col)
            l=df[col].unique()
            for mid in l:
                # if prev is not None:
        #             if prev['left'] != row['left']:
                # mid=float(row[col])/2
                subtableLeft=get_subtableSmaller(df,col,mid)
                subtableRight=get_subtableGreater(df,col,mid)
                fraction=len(subtableLeft)/len(df)
                fraction1=len(subtableRight)/len(df)
    #                 print(subtableLeft)
    #                 a=input()
                firstSubtable=0
                secondSubtable=0
                if subtableLeft.empty:
                    firstSubtable=0
                else:
                    firstSubtable=fraction*findParentEntropy(subtableLeft)
    #                 print(subtableRight)
                if subtableRight.empty:
                    secondSubtable=0
                else:
                    secondSubtable=fraction1*findParentEntropy(subtableRight)

                entropy2=firstSubtable+secondSubtable
                ig1=abs(entropy1)-abs(entropy2)

                if ig1>max1:
                    max1=ig1
                    value=mid
            # prev=row
            t=(max1,value)
            splitpoint[col]=t

        return splitpoint


    class node:
        def __init__(self,value):
            self.value=value
            self.child=[]
            self.edges=[]
            self.positive=0
            self.negative=0


    # In[190]:


    def buildTree(df):
        if len(df.columns)==1:
            return
        resultLabel= df.keys()[-1] 
        clValue,counts = np.unique(df[resultLabel],return_counts=True)

        #     print(clValue,counts)
        if(len(counts)==0):
            return 

        if len(counts)<=1:
            root=node(clValue[0])
            if clValue[0]==0:
                root.negative=counts[0]
            else:
                root.positive=counts[0]
        #         print("leaf")

        else:

            newsplit=split(df)
            calsplit=bestAttribute(df)
            newsplit.update(calsplit)
            maxig=0
            spoint=0
            attr=""
            for k,t1 in newsplit.items():
        #             t1=i.value()
                if maxig<t1[0]:
                    maxig=t1[0]
                    spoint=t1[1]
                    attr=k
            if attr=="":
                return None

            root=node(attr)
            root.split=spoint
            root.positive=counts[1]
            root.negative=counts[0]

            if attr in numeric:
                subtable0 = get_subtableSmaller(df,attr,spoint)
        #         subtable0 =subtable0.drop([attribute_name],axis=1)
                subtable1 = get_subtableGreater(df,attr,spoint)
                root.child.append(buildTree(subtable0))
                root.child.append(buildTree(subtable1))

            else:

                for j in df[attr].unique():
        #                     print(j)
                    subtable0 = get_subtable(df,attr,j)
                    subtable0 =subtable0.drop([attr],axis=1)
                    root.edges.append(j)
                    root.child.append(buildTree(subtable0))


        return root

    root=buildTree(df)
#     print(root.value)

    def testing(row1,root):
            if root==None:
                y1.append(0)
                return
            
            if len(root.child)==0:
                y1.append(root.value)
                return

            i=root.value
    #         if i in row1:
            num=0;
            if i in numeric:
                if row1[i]<root.split and root.child[0]!=None:
                    num=1;
                    testing(row1,root.child[0])
                elif row1[i]>=root.split and root.child[1]!=None:
                    num=1;
                    testing(row1,root.child[1])
                if num==0:
                    if root.positive>root.negative:
                        y1.append(1)
                    else:
                        y1.append(0)
                    return
                    
            else:
                k=0
                flag=0
                for j in root.edges:
                    if row1[i]==j and root.child[k]!=None:
#                         print("inside")
                        flag=1
                        testing(row1,root.child[k])
                        break
                    elif row1[i]==j:
                        if root.positive>root.negative:
                            y1.append(1)
                        else:
                            y1.append(0)
                        return
                    k=k+1
                if flag==0:
                    if root.positive>root.negative:
                        y1.append(1)
                    else:
                        y1.append(0)
                    return


    def testingData(df):
        for index,row in df.iterrows():
            testing(row,root)

    y1=[] 
    testingData(testXEntropy)  
#     print(len(y1))

    print(confusion_matrix(y_test, y1))  
    print(accuracy_score(y_test,y1))
    print(classification_report(y_test, y1)) 




In [7]:
def CR(OrignalXCR,OrignalYCR,testXCR,testYCR):
    def findParentCR(df):
        if df.empty==True:
            return
        classCol=df.keys()[-1]
        #yeh sirf header return karega
        resultValues=df[classCol].unique()
        #all ouput values in output column
        gini=math.inf
        for value in resultValues:
            fraction=df[classCol].value_counts()[value]/(len(df[classCol])+eps)
            gini=min(gini,fraction)
        return gini
    def findRelativeCR(df,attribute):
        if df.empty==True:
            return
        classCol= df.keys()[-1] 
        #last col assumed as result 
        #getting target class 
        resultValues=df[classCol].unique() 
        attributeNames=df[attribute].unique()
        gini2 = 0
        for attr in attributeNames:
            gini = math.inf
            for value in resultValues:
                #temp&hot with yes
                num = len(df[attribute][df[attribute]==attr][df[classCol]==value])
                #total hot
                den = len(df[attribute][df[attribute]==attr])
                fraction = num/(den+eps)
                gini =min(fraction,gini)
            #entropy attribute/total*(entropy attribute_values)
            fraction2 = den/len(df)  
            gini2 += fraction2*gini
        return abs(gini2)  
    def get_subtable(df, node,value):
        return df[df[node] == value].reset_index(drop=True)
    def get_subtableSmaller(df, node,value):
        if df.empty==True:
            return
        return df[df[node] < value].reset_index(drop=True)

    def get_subtableGreater(df, node,value):
        if df.empty==True:
            return
        return df[df[node] >= value].reset_index(drop=True)

    def bestAttributeCR(df):
        if df.empty==True:
            return
        infoGain={}
        max2=0
        attr=""
        entropy_1=findParentCR(df)
        for key in df.keys()[:-1]:

            if key not in numeric:
    #             print("XXX-------",key)
                entropy_2=findRelativeCR(df,key)
                if ((entropy_1-entropy_2)) > max2:
                    max2=entropy_1-entropy_2
                    attr=key
        t=(max2,0)
        infoGain[attr]=t
        return infoGain

    df=pd.concat([orignalXCR,orignalYCR],axis=1)
    def split(df):

        splitpoint={}
        entropy1=findParentCR(df)
        for col in numeric:
            # df.sort_values(col,inplace=True)
            prev=None;
            value=0;
            max1=0;
#             print(col)
            l=df[col].unique()
            for mid in l:
                # if prev is not None:
        #             if prev['left'] != row['left']:
                # mid=float(row[col])/2
                subtableLeft=get_subtableSmaller(df,col,mid)
                subtableRight=get_subtableGreater(df,col,mid)
                fraction=len(subtableLeft)/len(df)
                fraction1=len(subtableRight)/len(df)
    #                 print(subtableLeft)
    #                 a=input()
                firstSubtable=0
                secondSubtable=0
                if subtableLeft.empty:
                    firstSubtable=0
                else:
                    firstSubtable=fraction*findParentCR(subtableLeft)
    #                 print(subtableRight)
                if subtableRight.empty:
                    secondSubtable=0
                else:
                    secondSubtable=fraction1*findParentCR(subtableRight)

                entropy2=firstSubtable+secondSubtable
                ig1=abs(entropy1)-abs(entropy2)

                if ig1>max1:
                    max1=ig1
                    value=mid
            # prev=row
            t=(max1,value)
            splitpoint[col]=t

        return splitpoint


    class node:
        def __init__(self,value):
            self.value=value
            self.child=[]
            self.edges=[]
            self.positive=0
            self.negative=0


    # In[190]:


    def buildTree(df):
        if len(df.columns)==1:
            return
        resultLabel= df.keys()[-1] 
        clValue,counts = np.unique(df[resultLabel],return_counts=True)

    #     print(clValue,counts)
        if(len(counts)==0):
            return 
        
        if len(counts)<=1:
            root=node(clValue[0])
            if clValue[0]==0:
                root.negative=counts[0]
            else:
                root.positive=counts[0]
    #         print("leaf")

        else:

            newsplit=split(df)
            calsplit=bestAttributeCR(df)
            newsplit.update(calsplit)
            maxig=0
            spoint=0
            attr=""
            for k,t1 in newsplit.items():
    #             t1=i.value()
                if maxig<t1[0]:
                    maxig=t1[0]
                    spoint=t1[1]
                    attr=k
            if attr=="":
                return None
        
            root=node(attr)
            root.split=spoint
            root.positive=counts[1]
            root.negative=counts[0]

            if attr in numeric:
                subtable0 = get_subtableSmaller(df,attr,spoint)
        #         subtable0 =subtable0.drop([attribute_name],axis=1)
                subtable1 = get_subtableGreater(df,attr,spoint)
                root.child.append(buildTree(subtable0))
                root.child.append(buildTree(subtable1))

            else:

                for j in df[attr].unique():
#                     print(j)
                    subtable0 = get_subtable(df,attr,j)
                    subtable0 =subtable0.drop([attr],axis=1)
                    root.edges.append(j)
                    root.child.append(buildTree(subtable0))


        return root
    
    root=buildTree(df)
#     print(root.value)

    def testing(row1,root):
            if root==None:
                y1.append(0)
                return
            
            if len(root.child)==0:
                y1.append(root.value)
                return

            i=root.value
    #         if i in row1:
            num=0;
            if i in numeric:
                if row1[i]<root.split and root.child[0]!=None:
                    num=1;
                    testing(row1,root.child[0])
                elif row1[i]>=root.split and root.child[1]!=None:
                    num=1;
                    testing(row1,root.child[1])
                if num==0:
                    if root.positive>root.negative:
                        y1.append(1)
                    else:
                        y1.append(0)
                    return
                    
            else:
                k=0
                flag=0
                for j in root.edges:
                    if row1[i]==j and root.child[k]!=None:
#                         print("inside")
                        flag=1
                        testing(row1,root.child[k])
                        break
                    elif row1[i]==j:
                        if root.positive>root.negative:
                            y1.append(1)
                        else:
                            y1.append(0)
                        return
                    k=k+1
                if flag==0:
                    if root.positive>root.negative:
                        y1.append(1)
                    else:
                        y1.append(0)
                    return



    def testingData(df):
        for index,row in df.iterrows():
            testing(row,root)

    y1=[] 
    testingData(testXCR)  
#     print(len(y1))

    print(confusion_matrix(y_test, y1))  
    print(accuracy_score(y_test,y1))
    print(classification_report(y_test, y1)) 



    

In [8]:
y1=Entropy(orignalXEntropy,orignalYEntropy,testXEntropy,testYEntropy)
# print(confusion_matrix(y_test, y1))  
# print(accuracy_score(y_test,y1))
# print(classification_report(y_test, y1)) 

[[1685   36]
 [  20  507]]
0.9750889679715302
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      1721
           1       0.93      0.96      0.95       527

   micro avg       0.98      0.98      0.98      2248
   macro avg       0.96      0.97      0.97      2248
weighted avg       0.98      0.98      0.98      2248



In [9]:
Gini(orignalXGini,orignalYGini,testXGini,testYGini)


[[1693   28]
 [  46  481]]
0.9670818505338078
              precision    recall  f1-score   support

           0       0.97      0.98      0.98      1721
           1       0.94      0.91      0.93       527

   micro avg       0.97      0.97      0.97      2248
   macro avg       0.96      0.95      0.95      2248
weighted avg       0.97      0.97      0.97      2248



In [10]:
# print(confusion_matrix(y_test, y1))  
# print(accuracy_score(y_test,y1))
# print(classification_report(y_test, y1)) 

In [11]:
y1=CR(orignalXCR,orignalYCR,testXCR,testYCR)
# print(confusion_matrix(y_test, y1))  
# print(accuracy_score(y_test,y1))
# print(classification_report(y_test, y1)) 

[[1631   90]
 [  47  480]]
0.9390569395017794
              precision    recall  f1-score   support

           0       0.97      0.95      0.96      1721
           1       0.84      0.91      0.88       527

   micro avg       0.94      0.94      0.94      2248
   macro avg       0.91      0.93      0.92      2248
weighted avg       0.94      0.94      0.94      2248



In [12]:
# classifier = DecisionTreeClassifier()
# classifier.fit(orignalX, orignalY)  
# y_pred = classifier.predict(testX)  
# print(confusion_matrix(testY, y_pred))  
# print(classification_report(testY, y_pred)) 
# print(accuracy_score(testY,y_pred))